<a href="https://colab.research.google.com/github/kevin1109500/MACHINE-LEARNING/blob/main/HEALTHCARE_PROVIDER_FRAUD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from category_encoders.binary import BinaryEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from statistics import mean

Pre-processing Training data

In [ ]:
# #Load Train Dataset

Train=pd.read_csv("../input/healthcare-provider-fraud-detection-analysis/Train-1542865627584.csv")
Train_Beneficiarydata=pd.read_csv("../input/healthcare-provider-fraud-detection-analysis/Train_Beneficiarydata-1542865627584.csv")
Train_Inpatientdata=pd.read_csv("../input/healthcare-provider-fraud-detection-analysis/Train_Inpatientdata-1542865627584.csv")
Train_Outpatientdata=pd.read_csv("../input/healthcare-provider-fraud-detection-analysis/Train_Outpatientdata-1542865627584.csv")

# Load Test Dataset

Test=pd.read_csv("../input/healthcare-provider-fraud-detection-analysis/Test-1542969243754.csv")
Test_Beneficiarydata=pd.read_csv("../input/healthcare-provider-fraud-detection-analysis/Test_Beneficiarydata-1542969243754.csv")
Test_Inpatientdata=pd.read_csv("../input/healthcare-provider-fraud-detection-analysis/Test_Inpatientdata-1542969243754.csv")
Test_Outpatientdata=pd.read_csv("../input/healthcare-provider-fraud-detection-analysis/Test_Outpatientdata-1542969243754.csv")

In [ ]:
#merge all training data.
df_train = Train.merge(Train_Inpatientdata, how = 'left', on = 'Provider')
df_train = df_train.merge(Train_Outpatientdata, how = 'left', on = ['Provider', 'BeneID', 'ClaimID'])
df_train = df_train.merge(Train_Beneficiarydata, how = 'left', on = 'BeneID')

In [ ]:
#Creating new columns for number of claims
df_train[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']] = df_train[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']].where(~df_train[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']].notna(), 1)
df_train[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']] = df_train[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']].where(df_train[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']].notna(), 0)
df_train[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']] = df_train[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']].astype('int64')
claim_diagnose = []

for i in range(len(df_train)):
    claim_diagnose.append(df_train['ClmDiagnosisCode_1_x'][i]+ df_train['ClmDiagnosisCode_2_x'][i]+ df_train['ClmDiagnosisCode_3_x'][i]+ df_train['ClmDiagnosisCode_4_x'][i]+ df_train['ClmDiagnosisCode_5_x'][i]+ df_train['ClmDiagnosisCode_6_x'][i]+ df_train['ClmDiagnosisCode_7_x'][i]+df_train['ClmDiagnosisCode_8_x'][i]+  df_train['ClmDiagnosisCode_9_x'][i]+ df_train['ClmDiagnosisCode_10_x'][i])

df_train['num_claim_diagnose_in'] = claim_diagnose
df_train.drop(columns = ['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x'], axis = 1, inplace = True)
df_train[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']] = df_train[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']].where(~df_train[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']].notna(), 1)
df_train[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']] = df_train[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']].where(df_train[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']].notna(), 0)
df_train[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']] = df_train[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']].astype('int64')

claim_diagnose = []
for i in range(len(df_train)):
    claim_diagnose.append(df_train['ClmDiagnosisCode_1_y'][i]+ df_train['ClmDiagnosisCode_2_y'][i]+ df_train['ClmDiagnosisCode_3_y'][i]+ df_train['ClmDiagnosisCode_4_y'][i]+ df_train['ClmDiagnosisCode_5_y'][i]+ df_train['ClmDiagnosisCode_6_y'][i]+ df_train['ClmDiagnosisCode_7_y'][i]+df_train['ClmDiagnosisCode_8_y'][i]+  df_train['ClmDiagnosisCode_9_y'][i]+ df_train['ClmDiagnosisCode_10_y'][i])

df_train['num_claim_diagnose_out'] = claim_diagnose
df_train.drop(columns = ['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y'], axis = 1, inplace = True)

In [ ]:
df_train[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']] = df_train[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']].where(~df_train[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']].notna(), 1)
df_train[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']] = df_train[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']].where(df_train[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']].notna(), 0)
df_train[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']] = df_train[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']].astype('int64')
claim_procedure = []

for i in range(len(df_train)):
    claim_procedure.append(df_train['ClmProcedureCode_1_x'][i]+ df_train['ClmProcedureCode_2_x'][i]+ df_train['ClmProcedureCode_3_x'][i]+ df_train['ClmProcedureCode_4_x'][i]+ df_train['ClmProcedureCode_5_x'][i])

df_train['num_claim_procedure_in'] = claim_procedure
df_train.drop(columns = ['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x'], axis = 1, inplace = True)
df_train[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']] = df_train[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']].where(~df_train[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']].notna(), 1)
df_train[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']] = df_train[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']].where(df_train[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']].notna(), 0)
df_train[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']] = df_train[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']].astype('int64')
claim_procedure = []

for i in range(len(df_train)):
    claim_procedure.append(df_train['ClmProcedureCode_1_y'][i]+ df_train['ClmProcedureCode_2_y'][i]+ df_train['ClmProcedureCode_3_y'][i]+ df_train['ClmProcedureCode_4_y'][i]+ df_train['ClmProcedureCode_5_y'][i])

df_train['num_claim_procedure_out'] = claim_procedure
df_train.drop(columns = ['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y'], axis = 1, inplace = True)

In [ ]:
#Mapping common physicians
df_train[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']] = df_train[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']].where(~df_train[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']].notna(), 1)
df_train[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']] = df_train[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']].where(df_train[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']].notna(), 0)
df_train[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']] = df_train[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']].where(~df_train[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']].notna(), 1)
df_train[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']] = df_train[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']].where(df_train[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']].notna(), 0)

df_train[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']] = df_train[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']].astype('int64')
df_train[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']] = df_train[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']].astype('int64')

In [ ]:
df_train = df_train.replace({'ChronicCond_Alzheimer': 2, 'ChronicCond_Heartfailure': 2, 'ChronicCond_KidneyDisease': 2,
                           'ChronicCond_Cancer': 2, 'ChronicCond_ObstrPulmonary': 2, 'ChronicCond_Depression': 2,
                           'ChronicCond_Diabetes': 2, 'ChronicCond_IschemicHeart': 2, 'ChronicCond_Osteoporasis': 2,
                           'ChronicCond_rheumatoidarthritis': 2, 'ChronicCond_stroke': 2 }, 0)

In [ ]:
#Dropping unneccessary columns
df_train = df_train.groupby(['Provider', 'PotentialFraud'],as_index=False).agg('sum')
df_train.drop(columns = ['Gender', 'Race', 'State', 'County'], inplace = True, axis = 1)

In [ ]:
#Renaming columns.
name = []
temp = []
for column in df_train.columns:
    if column[-2:] == '_x':
        name.append(column)
        temp.append(column[:-2] + '_In')

for i in range(len(name)):
    df_train=df_train.rename(columns = {name[i]:temp[i]})

name = []
temp = []
for column in df_train.columns:
    if column[-2:] == '_y':
        name.append(column)
        temp.append(column[:-2] + '_Out')

for i in range(len(name)):
    df_train=df_train.rename(columns = {name[i]:temp[i]})

pd.set_option('display.max_columns', None)

In [ ]:
#Columns that have only one value are dropped.
deleted = []
for feature in df_train.columns:
    if df_train[feature].nunique() <= 1:
        deleted.append(feature)

df_train = df_train[[i for i in df_train.columns if i not in deleted]]

Pre-processing Testing data similarly

In [ ]:
df_test = Test.merge(Test_Inpatientdata, how = 'left', on = 'Provider')
df_test = df_test.merge(Test_Outpatientdata, how = 'left', on = ['Provider', 'BeneID', 'ClaimID'])
df_test = df_test.merge(Test_Beneficiarydata, how = 'left', on = 'BeneID')

In [ ]:
df_test[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']] = df_test[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']].where(~df_test[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']].notna(), 1)
df_test[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']] = df_test[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']].where(df_test[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']].notna(), 0)
df_test[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']] = df_test[['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x']].astype('int64')

claim_diagnose = []

for i in range(len(df_test)):
    claim_diagnose.append(df_test['ClmDiagnosisCode_1_x'][i]+ df_test['ClmDiagnosisCode_2_x'][i]+ df_test['ClmDiagnosisCode_3_x'][i]+ df_test['ClmDiagnosisCode_4_x'][i]+ df_test['ClmDiagnosisCode_5_x'][i]+ df_test['ClmDiagnosisCode_6_x'][i]+ df_test['ClmDiagnosisCode_7_x'][i]+df_test['ClmDiagnosisCode_8_x'][i]+  df_test['ClmDiagnosisCode_9_x'][i]+ df_test['ClmDiagnosisCode_10_x'][i])

df_test['num_claim_diagnose_in'] = claim_diagnose

df_test.drop(columns = ['ClmDiagnosisCode_1_x', 'ClmDiagnosisCode_2_x', 'ClmDiagnosisCode_3_x', 'ClmDiagnosisCode_4_x', 'ClmDiagnosisCode_5_x', 'ClmDiagnosisCode_6_x', 'ClmDiagnosisCode_7_x', 'ClmDiagnosisCode_8_x', 'ClmDiagnosisCode_9_x', 'ClmDiagnosisCode_10_x'], axis = 1, inplace = True)
df_test[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']] = df_test[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']].where(~df_test[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']].notna(), 1)
df_test[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']] = df_test[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']].where(df_test[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']].notna(), 0)
df_test[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']] = df_test[['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y']].astype('int64')

claim_diagnose = []

for i in range(len(df_test)):
    claim_diagnose.append(df_test['ClmDiagnosisCode_1_y'][i]+ df_test['ClmDiagnosisCode_2_y'][i]+ df_test['ClmDiagnosisCode_3_y'][i]+ df_test['ClmDiagnosisCode_4_y'][i]+ df_test['ClmDiagnosisCode_5_y'][i]+ df_test['ClmDiagnosisCode_6_y'][i]+ df_test['ClmDiagnosisCode_7_y'][i]+df_test['ClmDiagnosisCode_8_y'][i]+  df_test['ClmDiagnosisCode_9_y'][i]+ df_test['ClmDiagnosisCode_10_y'][i])

df_test['num_claim_diagnose_out'] = claim_diagnose
df_test.drop(columns = ['ClmDiagnosisCode_1_y', 'ClmDiagnosisCode_2_y', 'ClmDiagnosisCode_3_y', 'ClmDiagnosisCode_4_y', 'ClmDiagnosisCode_5_y', 'ClmDiagnosisCode_6_y', 'ClmDiagnosisCode_7_y', 'ClmDiagnosisCode_8_y', 'ClmDiagnosisCode_9_y', 'ClmDiagnosisCode_10_y'], axis = 1, inplace = True)


In [ ]:
df_test[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']] = df_test[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']].where(~df_test[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']].notna(), 1)
df_test[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']] = df_test[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']].where(df_test[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']].notna(), 0)
df_test[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']] = df_test[['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x']].astype('int64')
claim_procedure = []

for i in range(len(df_test)):
    claim_procedure.append(df_test['ClmProcedureCode_1_x'][i]+ df_test['ClmProcedureCode_2_x'][i]+ df_test['ClmProcedureCode_3_x'][i]+ df_test['ClmProcedureCode_4_x'][i]+ df_test['ClmProcedureCode_5_x'][i])

df_test['num_claim_procedure_in'] = claim_procedure
df_test.drop(columns = ['ClmProcedureCode_1_x', 'ClmProcedureCode_2_x', 'ClmProcedureCode_3_x', 'ClmProcedureCode_4_x', 'ClmProcedureCode_5_x'], axis = 1, inplace = True)
df_test[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']] = df_test[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']].where(~df_test[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']].notna(), 1)
df_test[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']] = df_test[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']].where(df_test[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']].notna(), 0)
df_test[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']] = df_test[['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y']].astype('int64')

claim_procedure = []

for i in range(len(df_test)):
    claim_procedure.append(df_test['ClmProcedureCode_1_y'][i]+ df_test['ClmProcedureCode_2_y'][i]+ df_test['ClmProcedureCode_3_y'][i]+ df_test['ClmProcedureCode_4_y'][i]+ df_test['ClmProcedureCode_5_y'][i])

df_test['num_claim_procedure_out'] = claim_procedure
df_test.drop(columns = ['ClmProcedureCode_1_y', 'ClmProcedureCode_2_y', 'ClmProcedureCode_3_y', 'ClmProcedureCode_4_y', 'ClmProcedureCode_5_y'], axis = 1, inplace = True)

In [ ]:
df_test[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']] = df_test[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']].where(~df_test[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']].notna(), 1)
df_test[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']] = df_test[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']].where(df_test[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']].notna(), 0)
df_test[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']] = df_test[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']].where(~df_test[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']].notna(), 1)
df_test[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']] = df_test[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']].where(df_test[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']].notna(), 0)
df_test[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']] = df_test[['AttendingPhysician_x', 'OperatingPhysician_x', 'OtherPhysician_x']].astype('int64')
df_test[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']] = df_test[['AttendingPhysician_y', 'OperatingPhysician_y', 'OtherPhysician_y']].astype('int64')

In [ ]:
df_test = df_test.replace({'ChronicCond_Alzheimer': 2, 'ChronicCond_Heartfailure': 2, 'ChronicCond_KidneyDisease': 2,
                           'ChronicCond_Cancer': 2, 'ChronicCond_ObstrPulmonary': 2, 'ChronicCond_Depression': 2,
                           'ChronicCond_Diabetes': 2, 'ChronicCond_IschemicHeart': 2, 'ChronicCond_Osteoporasis': 2,
                           'ChronicCond_rheumatoidarthritis': 2, 'ChronicCond_stroke': 2 }, 0)

In [ ]:
df_test = df_test.groupby(['Provider'],as_index=False).agg('sum')
df_test.drop(columns = ['Gender', 'Race', 'State', 'County'], inplace = True, axis = 1)

In [ ]:
name = []
temp = []
for column in df_test.columns:
    if column[-2:] == '_x':
        name.append(column)
        temp.append(column[:-2] + '_In')

for i in range(len(name)):
    df_test=df_test.rename(columns = {name[i]:temp[i]})

name = []
temp = []
for column in df_test.columns:
    if column[-2:] == '_y':
        name.append(column)
        temp.append(column[:-2] + '_Out')

for i in range(len(name)):
    df_test=df_test.rename(columns = {name[i]:temp[i]})

deleted = []
for feature in df_test.columns:
    if df_test[feature].nunique() <= 1:
        deleted.append(feature)

df_test = df_test[[i for i in df_test.columns if i not in deleted]]

In [ ]:
## Fill missing values

df_m = df_train.copy()

num_cols = [i for i in df_m.columns if df_m[i].dtypes == 'float64' or df_m[i].dtypes == 'int64']


for i in num_cols:
    df_m[i].fillna(0, inplace=True)

In [ ]:
#Select columns using correlation between categorical and numerical data.
df_m["PotentialFraud"] = df_m["PotentialFraud"].map({"Yes" : 1,"No" : 0})
num_cols = num_cols + ["PotentialFraud"]
corr = df_m[num_cols].corr(method='kendall')['PotentialFraud'].reset_index().rename(columns = {'index' : 'feature'})

select_cols = []
for i in corr['feature'].unique():
    if (corr.loc[corr.feature == i, 'PotentialFraud'].values[0] >= 0.01) | (corr.loc[corr.feature == i, 'PotentialFraud'].values[0] <= -0.01):
        select_cols.append(i)
    else:
        continue

In [ ]:
#Use columns to be features of train data.
df_temp = df_train[select_cols]

df_temp = df_temp.loc[:,~df_temp.columns.duplicated()]

In [ ]:
# split data into features and target and then train and test.
df_temp["PotentialFraud"] = df_temp["PotentialFraud"].map({"Yes" : 1,"No" : 0})
X_train_val = df_temp.drop(columns = "PotentialFraud")
y_train_val = df_temp["PotentialFraud"]


#Use columns to be features of test data.
select_test = [i for i in select_cols if i != 'PotentialFraud'] + ["Provider"]
test = df_test[select_test]
test = test.loc[:,~test.columns.duplicated()]

id_test = test[["Provider"]]
X_input = test.drop(columns = ["Provider"], axis = 1)

In [ ]:
#Scaling data
num_cols = [i for i in X_train_val.columns if X_train_val[i].dtypes == 'float64' or X_train_val[i].dtypes == 'int64']

num_transform = Pipeline([("scaler", StandardScaler())])

prep_pipeline =  ColumnTransformer([('num trans', num_transform, num_cols)])

Fitting algorithms

In [ ]:
model_lr = Pipeline([
    ('prep', prep_pipeline),
    ('lr',LogisticRegression(random_state = 0))
])

model_rf = Pipeline([
    ('prep', prep_pipeline),
    ('rf',RandomForestClassifier(random_state = 0))
])

model_xg = Pipeline([
    ('prep', prep_pipeline),
    ('xg',XGBClassifier(random_state = 0))
])

model_dt = Pipeline([
    ('prep', prep_pipeline),
    ('dt',DecisionTreeClassifier(random_state = 0))
])

model_svc = Pipeline([
    ('prep', prep_pipeline),
    ('svc',SVC(random_state=0))
])

model_knn = Pipeline([
    ('prep', prep_pipeline),
    ('knn',KNeighborsClassifier())
])

In [ ]:
def model_evaluation_cv(model, metric):
    model_cv = cross_val_score(model, X_train_val, y_train_val, cv = 5, scoring = metric)
    return model_cv

print('Accuracy Score Evaluation of SVM : ', mean(model_evaluation_cv(model_svc, 'accuracy')))
print('Precision Score Evaluation of SVM : ', mean(model_evaluation_cv(model_svc, 'precision')))
print('Recall Score Evaluation of SVM : ', mean(model_evaluation_cv(model_svc, 'recall')))
print('\n')
print('Accuracy Score Evaluation of KNN : ', mean(model_evaluation_cv(model_knn, 'accuracy')))
print('Precision Score Evaluation of KNN : ', mean(model_evaluation_cv(model_knn, 'precision')))
print('Recall Score Evaluation of KNN : ', mean(model_evaluation_cv(model_knn, 'recall')))
print('\n')
print('Accuracy Score Evaluation of Random Forest : ', mean(model_evaluation_cv(model_rf, 'accuracy')))
print('Precision Score Evaluation of Random Forest : ', mean(model_evaluation_cv(model_rf, 'precision')))
print('Recall Score Evaluation of Random Forest : ', mean(model_evaluation_cv(model_rf, 'recall')))
print('\n')
print('Accuracy Score Evaluation of XGBoost : ', mean(model_evaluation_cv(model_xg, 'accuracy')))
print('Precision Score Evaluation of XGBoost : ', mean(model_evaluation_cv(model_xg, 'precision')))
print('Recall Score Evaluation of XGBoost : ', mean(model_evaluation_cv(model_xg, 'recall')))
print('\n')
print('Accuracy Score Evaluation of Decision Tree : ', mean(model_evaluation_cv(model_dt, 'accuracy')))
print('Precision Score Evaluation of Decision Tree : ', mean(model_evaluation_cv(model_dt, 'precision')))
print('Recall Score Evaluation of Decision Tree : ', mean(model_evaluation_cv(model_dt, 'recall')))
print('\n')
print('Accuracy Score Evaluation of Logistic Regression : ', mean(model_evaluation_cv(model_lr, 'accuracy')))
print('Precision Score Evaluation of Logistic Regression : ', mean(model_evaluation_cv(model_lr, 'precision')))
print('Recall Score Evaluation of Logistic Regression : ', mean(model_evaluation_cv(model_lr, 'recall')))

Accuracy Score Evaluation of SVM :  0.9266173752310536
Precision Score Evaluation of SVM :  0.7639244026934484
Recall Score Evaluation of SVM :  0.32230634828188703


Accuracy Score Evaluation of KNN :  0.9177449168207024
Precision Score Evaluation of KNN :  0.6097977114961181
Recall Score Evaluation of KNN :  0.3402057852844108


Accuracy Score Evaluation of Random Forest :  0.9245841035120148
Precision Score Evaluation of Random Forest :  0.6878289982441297
Recall Score Evaluation of Random Forest :  0.379557367501456


Accuracy Score Evaluation of XGBoost :  0.9216266173752311
Precision Score Evaluation of XGBoost :  0.6291086023808372
Recall Score Evaluation of XGBoost :  0.41122112211221123


Accuracy Score Evaluation of Decision Tree :  0.9029574861367837
Precision Score Evaluation of Decision Tree :  0.48598929849031713
Recall Score Evaluation of Decision Tree :  0.43292564550572704


Accuracy Score Evaluation of Logistic Regression :  0.9288354898336414
Precision Score Evaluati